<a href="https://colab.research.google.com/github/bemakerorg/AIoT_Book_RF/blob/main/AIoT_RF_Book_ES_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ESERCIZIO 11**
Iniziamo importando alcune librerie Python.

In [ ]:
# requests: Libreria per effettuare richieste HTTP
import requests
# Dalla Libreria Pillow per l’elaborazione immagini, viene caricato il modulo image
from PIL import Image
# BytesIO: per gestire l’immagine in binario
from io import BytesIO

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# URL dell'immagine su Pixabay
image_url = 'https://cdn.pixabay.com/photo/2016/12/29/15/06/tower-bridge-1938684_960_720.jpg'

# Scaricare l'immagine usando requests
response = requests.get(image_url)
if response.status_code == 200:
    image = Image.open(BytesIO(response.content))

    # Visualizzare l'immagine originale
    plt.imshow(image)
    plt.axis('off')
    plt.show()
else: # gestisce l'eventuale errore della richiesta HTTP
    print("Errore nel download dell'immagine:", response.status_code)

In [ ]:
# Ridimensionare l'immagine a 512x512 pixel
resized_image = image.resize((512, 512))

# Convertire l'immagine ridimensionata in scala di grigi (bianco e nero)
gray_resized_image = resized_image.convert('L')

# Convertire l'immagine in un array NumPy
gray_image_array = np.array(gray_resized_image)

In [ ]:
# Visualizzare l'immagine ridimensionata e in scala di grigi
plt.figure(figsize=(48, 24)) #dimensione finestra immagine

# Immagine ridimensionata
plt.subplot(1, 2, 1)
plt.imshow(resized_image)
plt.title('Immagine ridimensionata (512x512)', fontsize=40)
plt.axis('off')

# Immagine ridimensionata in scala di grigi
plt.subplot(1, 2, 2)
plt.imshow(gray_resized_image, cmap='gray')
plt.title('Immagine in bianco e nero (512x512)', fontsize=40)
plt.axis('off')

In [ ]:
# Funzione per applicare il filtro di Sobel a un'immagine
def apply_filter(image_array, filter):
    size_x, size_y = image_array.shape
    transformed_image = np.copy(image_array)
    weight = 1

    for x in range(1, size_x - 1):
        for y in range(1, size_y - 1):
            convolution = 0.0
            convolution += (image_array[x - 1, y - 1] * filter[0, 0])
            convolution += (image_array[x, y - 1] * filter[1, 0])
            convolution += (image_array[x + 1, y - 1] * filter[2, 0])
            convolution += (image_array[x - 1, y] * filter[0, 1])
            convolution += (image_array[x, y] * filter[1, 1])
            convolution += (image_array[x + 1, y] * filter[2, 1])
            convolution += (image_array[x - 1, y + 1] * filter[0, 2])
            convolution += (image_array[x, y + 1] * filter[1, 2])
            convolution += (image_array[x + 1, y + 1] * filter[2, 2])
            convolution *= weight
            if convolution < 0:
                convolution = 0
            if convolution > 255:
                convolution = 255
            transformed_image[x, y] = convolution

    return transformed_image

In [ ]:
# Definizione Filtri di Sobel [3x3]
horizontal_filter = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])
vertical_filter = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])

# Applica i filtri
vertical_edges = apply_filter(gray_image_array, vertical_filter)
horizontal_edges = apply_filter(gray_image_array, horizontal_filter)

In [ ]:
plt.figure(figsize=(48, 24)) #dimensione finestra immagine

# Immagine con filtro per linee verticali
plt.subplot(1, 2, 2)
plt.imshow(vertical_edges, cmap='gray')
plt.title('Immagine con filtro linee verticali', fontsize=40)
plt.axis('off')

# Immagine con filtro per linee orizzontali
plt.subplot(1, 2, 1)
plt.imshow(horizontal_edges, cmap='gray')
plt.title('Immagine con filtro linee orizzontali', fontsize=40)
plt.axis('off')

In [ ]:
# Funzione per comprimere l'immagine
def compress_image(image_array):
    size_x, size_y = image_array.shape
    new_x = int(size_x / 4)
    new_y = int(size_y / 4)
    new_image = np.zeros((new_x, new_y))

    for x in range(0, size_x, 4):
        for y in range(0, size_y, 4):
            pixels = []
            for dx in range(4):
                for dy in range(4):
                    pixels.append(image_array[x + dx, y + dy])
            pixels.sort(reverse=True)
            new_image[int(x / 4), int(y / 4)] = pixels[0]

    return new_image

In [ ]:
# Comprimere le immagini filtrate
compressed_vertical_edges = compress_image(vertical_edges)
compressed_horizontal_edges = compress_image(horizontal_edges)

# Visualizzare le immagine compresse
plt.figure(figsize=(24, 12))

# Immagine compressa con filtro verticale
plt.subplot(1, 2, 2)
plt.imshow(compressed_vertical_edges, cmap='gray')
plt.title('Immagine compressa (filtro verticale)', fontsize=20)
plt.axis('off')

# Immagine compressa con filtro orizzontale
plt.subplot(1, 2, 1)
plt.imshow(compressed_horizontal_edges, cmap='gray')
plt.title('Immagine compressa (filtro orizzontale)', fontsize=20)
plt.axis('off')

plt.show()